In [96]:
import os
import cv2
import torch 
import numpy as np
import time
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
# init license plate model
licensePlateModel = torch.hub.load('../yolov5', 'custom', path='../models/best_lp_4.pt', source='local').to(device)  # local repo
licensePlateModel.eval()

# init license plate model
model_detetect_number = torch.hub.load('../yolov5', 'custom', path='../models/detect_number_3.pt', source='local').to(device)  # local repo
model_detetect_number.eval()

YOLOv5  v6.1-132-g014acde torch 1.11.0+cu113 CUDA:0 (Quadro M500M, 2048MiB)

Fusing layers... 
Model summary: 213 layers, 1760518 parameters, 0 gradients
Adding AutoShape... 
YOLOv5  v6.1-132-g014acde torch 1.11.0+cu113 CUDA:0 (Quadro M500M, 2048MiB)

Fusing layers... 
Model summary: 213 layers, 1807873 parameters, 0 gradients
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): Model(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 16, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1))
           

In [97]:
def applySummFunctin(img, axis = 1):
    res = np.sum(img, axis = axis) 
    return res

def splitLine(img):
    if (img.shape[0]/img.shape[1])<(1/3):
        return None

    s1 = applySummFunctin(img, axis =1)
    lst_max = np.where(s1== max(s1))[0]
    deadline = lst_max[int(len(lst_max)/2)]
    if deadline<(1/3)*len(s1) - deadline or deadline>(1/3)*len(s1) - deadline :
        deadline = int(len(s1)/2)    
    return deadline


def sort(bboxes, deadline):
    if deadline == None:
        return bboxes[np.argsort(bboxes[:,-2])]
    else:
        l1 = bboxes[bboxes[:,-1]<deadline]
        l2 = bboxes[bboxes[:,-1]>deadline]
        return np.concatenate((l1[np.argsort(l1[:,-2])], l2[np.argsort(l2[:,-2])]))


def get_license_plate(image ):

        bboxes = licensePlateModel(image).pandas().xyxy[0].to_numpy()
        if len(bboxes)==0:
            return None, 'Cant detect vehicle !'
        bboxes = bboxes[bboxes[:,5]==max(bboxes[:,5])][0]
        xmin, ymin, xmax, ymax = bboxes[0:4]
        img_license_plate = image[int(ymin):int(ymax), int(xmin):int(xmax)]
        
        return img_license_plate, None

def recognize_plate(img_plate):
    img_plate =cv2.cvtColor(img_plate, cv2.COLOR_BGR2GRAY)#COLOR_BGR2GRAY
    bboxes = model_detetect_number(img_plate).pandas().xyxy[0].to_numpy()
    bboxes = bboxes[bboxes[:,4]>0.65]
    
    bboxes = np.append(bboxes, (bboxes[:,0]+(bboxes[:,2]-bboxes[:,0])/2).reshape(-1,1), axis = 1)
    bboxes =  np.append(bboxes, (bboxes[:,1]+(bboxes[:,3]-bboxes[:,1])/2).reshape(-1,1), axis = 1)
    deadline = splitLine(img_plate)
    return ''.join(sort(bboxes, deadline)[:,6])

def Full_Detect(image):
    img_plate,err= get_license_plate(image)
    if err != None:
        return 'can\'t detection'   
    return recognize_plate(img_plate)

In [98]:
path_folder = r'E:\IAMDAT\Workplace\DOANCUOIKY\DesktopApp\core\images\images_test'
lis_img =  os.listdir(path_folder)

In [99]:
count = 0
t=time.time()
for i in lis_img:

    path = os.path.join(path_folder, i)
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    if Full_Detect(img) == i.split('.')[0] or i.split('.')[0] == 'no' :
        # break
        count+=1

print('image/time = ',(time.time()-t)/len(lis_img))

image/time =  0.07882682176736686


In [100]:
count

34

In [14]:
34/52

0.6538461538461539

In [106]:
import logging
from sklearn.cluster import KMeans

from torchvision import transforms

from torch import nn
import torch.nn.functional as F
from PIL import Image
transform = transforms.Compose([transforms.ToTensor()])

# def get_char():
    


device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
licensePlateModel = torch.hub.load(r'E:\IAMDAT\Workplace\DOANCUOIKY\ok\yolov5', 'custom', path=r'E:\IAMDAT\Workplace\DOANCUOIKY\FinalVersion\models\best_lp_2.pt', source='local').to(device)


img = cv2.imread(r'E:\IAMDAT\Workplace\DOANCUOIKY\DesktopApp\core\images\vehicle\1649952038.7674325.jpg')

s = '0123456789ABCDEFGHKLMNPRSTUVXYZ'
i2c = {len(s):'None'}
c2i = {'None':len(s)}
for i,v in enumerate(s):
  i2c[i] = v
  c2i[v] = i

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        #40 40
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.fc1 = nn.Linear(16 * 8 * 8, 120)
        self.fc2 = nn.Linear(120, 31)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))#6 20 20
        x = self.pool(F.relu(self.conv2(x)))# 16 10 10
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
net = Net().to(device)
net.load_state_dict(torch.load(r'E:\IAMDAT\Workplace\DOANCUOIKY\FinalVersion\models\classification1.pth'))
net.eval()




def GDS(arr):
    gds = []
    mean = np.mean(arr)
    std = np.std(arr)
    th=3
    p_max, p_min = mean+std, mean-std
    for i in range(th, len(arr)-th):
        if ((arr[i]<np.mean(arr[i-th:i]) and arr[i]>np.mean(arr[i:i+th])) or (arr[i]>np.mean(arr[i-th:i]) and arr[i]<np.mean(arr[i:i+th]))) and arr[i]<= p_max and arr[i]>=p_min:
            gds.append(i)
    return gds

def applySummFunctin(img, axis = 1):
    res = np.sum(img, axis = axis)    #  summ elements in columns
    return res

def get_plate(img):
    err=  None
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    bbox = licensePlateModel(img).pandas().xyxy[0].to_numpy()
    if len(bbox) == 0:
        err = 'Can\'t detect license plate'
        logging.exception(err)
        return None, err
    bbox = bbox[bbox[:,5]==np.max(bbox[:,5])][0]
    xmin, ymin, xmax, ymax = bbox[0:4]
    img_license_plate = img[int(ymin):int(ymax), int(xmin):int(xmax)]	
    return img_license_plate, err





def change_to_binary(img):
    kmeans = KMeans(n_clusters=2, random_state=0).fit(img.reshape((-1,1)))
    predict = kmeans.predict(img.reshape((-1,1))).reshape(img.shape)
    centers = kmeans.cluster_centers_
    if centers[0]>centers[1]:
        predict = 1-predict
    return predict

# def splitLine(img):
    
#     predict = change_to_binary(img)
#     s1 = applySummFunctin(img, axis =1)
#     lst_max = np.where(s1== max(s1))[0]
#     deadline = lst_max[int(len(lst_max)/2)]
#     line1_binary = predict[0:deadline,:]
#     line2_binary = predict[deadline:-1,:]
#     line1  = img[0:deadline,:]
#     line2 = img[deadline:-1,:]
#     return [line1, line2],[line1_binary, line2_binary]


def splitLine(img):
    kmeans = KMeans(n_clusters=2, random_state=0).fit(img.reshape((-1,1)))
    predict = kmeans.predict(img.reshape((-1,1))).reshape(img.shape)
    centers = kmeans.cluster_centers_
    if centers[0]>centers[1]:
        predict = 1-predict

    s1 = applySummFunctin(img, axis =1)
    lst_max = np.where(s1== max(s1))[0]
    deadline = lst_max[int(len(lst_max)/2)]
    if deadline<(1/3)*len(s1) - deadline or deadline>(1/3)*len(s1) - deadline :
        deadline = int(len(s1)/2)

    
    

    line1 = img[0:deadline,:]
    sum01 = applySummFunctin(line1, axis =0)
    gds01 = GDS(sum01)
    line1 = line1[ :, gds01[0]:gds01[-1]]

    line2 = img[deadline:-1,:]
    sum02 = applySummFunctin(line2, axis =0)
    gds02 = GDS(sum02)
    line2 = line2[ :, gds02[0]:gds02[-1]]



    line1_binary = predict[0:deadline,gds01[0]:gds01[-1]]
    line2_binary = predict[deadline:-1,gds02[0]:gds02[-1]]
    return [line1, line2],[line1_binary, line2_binary]


def get_location_of_char(line):

    arr = applySummFunctin(line, axis =0)
    locations = []
    lst = []

    mean = np.mean(arr)
    arr_tf =  arr>= mean 
    for i in range(len(arr_tf)):
        if arr_tf[i]==False and i!=len(arr_tf)-1:
            lst.append(i)

        elif arr_tf[i]==True:
            if  i!=len(arr_tf)-1 and arr_tf[i+1]==False and len(lst)<len(arr)/10:
                continue
            if len(lst)<4:
                lst = []
            else:
                locations.append([lst[0], lst[-1]])
                lst = []
        else:
            lst.append(i)
            locations.append([lst[0], lst[-1]])
            lst = []

    return locations

def getCharInPlate(img):
    chars= []
    plate, err = get_plate(img)
    if err != None:
        return chars
    sum_1 = applySummFunctin(plate, axis =1)
    # sum_0 = applySummFunctin(plate, axis =0)
    gds1 = GDS(sum_1)
    # gds0 = GDS(sum_0)
    sub = plate[ gds1[0]:gds1[-1],:]
    if (sub.shape[0]/sub.shape[1])>(1/3):
        lines, lines_b = splitLine(sub)
        
    else:
        lines, lines_b = [sub],[change_to_binary(sub)]
    for ind in range(len(lines)):
        # s0 = applySummFunctin(, axis =0)

        gds = get_location_of_char(lines_b[ind])
        for i in gds:
            if i[0]>2:
                img_line = lines[ind][:,i[0]-2:i[1]+2]
            else:
                img_line = lines[ind][:,i[0]:i[1]]
            # scale_percent = 400 # percent of original size
            # width = int(img_line.shape[1] * scale_percent / 100)
            # height = int(img_line.shape[0] * scale_percent / 100)
            dim = (40, 40)
            # resize image
            resized = cv2.resize(img_line, dim, interpolation = cv2.INTER_AREA)
            chars.append(resized)

    chars = np.array(chars)/255.0
    tenso = torch.from_numpy(chars.reshape(-1,1,40,40)).to(device).float()
    _,predicted = torch.max(net(tenso),1)

    result = ''
    for i in predicted:
        result+= i2c[i.item()]
    return result


YOLOv5  v6.1-132-g014acde torch 1.11.0+cu113 CUDA:0 (Quadro M500M, 2048MiB)

Fusing layers... 
Model summary: 213 layers, 1760518 parameters, 0 gradients
Adding AutoShape... 


In [107]:
getCharInPlate(img)

'59S28479'

In [108]:
count = 0
t=time.time()
for i in lis_img:

    path = os.path.join(path_folder, i)
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    if getCharInPlate(img) == i.split('.')[0] or i.split('.')[0] == 'no' :
        # break
        count+=1

print('image/time = ',(time.time()-t)/len(lis_img))

Can't detect license plate
NoneType: None


image/time =  0.09896150460610023


In [109]:
count

6

In [110]:
len(lis_img)

52

In [113]:
6/50

0.12

In [118]:
36/50

0.72

In [122]:
9/50

0.18

In [ ]:
import imutils
import pytesseract
pytesseract.pytesseract.tesseract_cmd = 'C:/Program Files/Tesseract-OCR/tesseract.exe'
def getLicensePlate_motorcycle(self, sub_img):
        image = cv2.imread(sub_img)

        image = imutils.resize(image, width=300)

        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        gray_image = cv2.bilateralFilter(gray_image, 11, 17, 17)

        edged = cv2.Canny(gray_image, 30, 200)

        cnts,new = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

        cnts = sorted(cnts, key = cv2.contourArea, reverse = True) [:5]

        screenCnt = []
        i = 0

        for c in cnts:
            perimeter = cv2.arcLength(c, True)
            approx = cv2.approxPolyDP(c, 0.018 * perimeter, True)
            if len(approx) == 4: 
                screenCnt.append(approx)
                x,y,w,h = cv2.boundingRect(c)
                new_img=image[y:y+h,x:x+w]
                hh = int(h/2)
                line1 = image[y:y+hh, x:x+w]
                line1 = imutils.resize(line1, width=300)
                line2 = image[y+hh:y+h, x:x+w]
                cv2.imwrite('./images/temp_license_plate/'+str(i)+'.png',new_img)
                cv2.imwrite('./images/temp_license_plate/'+str(i)+'.1.png',line1)
                cv2.imwrite('./images/temp_license_plate/'+str(i)+'.2.png',line2)
                i += 1
        for j in range(0, i, 1):
            Cropped_loc = "./images/temp_license_plate/" + str(j) + ".png"
            line1 = "./images/temp_license_plate/" + str(j) + ".1.png"
            line2 = "./images/temp_license_plate/" + str(j) + ".2.png"
            plate1 = pytesseract.image_to_string(line1, lang='eng', config ='--oem 3 --psm 7 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789')
            plate2 = pytesseract.image_to_string(line2, lang='eng', config ='--oem 3 --psm 7 -c tessedit_char_whitelist=0123456789')
            plate = plate1+plate2
            # plate = plate.replace('\n', "")
            # if(5 <= len(plate) <=9):
            return plate

    def getLicensePlate_car_truck(self, sub_img):
        image = cv2.imread(sub_img)

        image = imutils.resize(image, width=500)

        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        gray_image = cv2.bilateralFilter(gray_image, 11, 17, 17)

        edged = cv2.Canny(gray_image, 30, 200)

        cnts,new = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

        cnts = sorted(cnts, key = cv2.contourArea, reverse = True) [:5]

        screenCnt = []
        i = 0
        flag = []

        for c in cnts:
            perimeter = cv2.arcLength(c, True)
            approx = cv2.approxPolyDP(c, 0.018 * perimeter, True)
            if len(approx) == 4: 
                screenCnt.append(approx)
                x,y,w,h = cv2.boundingRect(c)
                print(h)
                if(h < 30):
                    new_img=image[y:y+h,x:x+w]
                    cv2.imwrite('./images/temp_license_plate/'+str(i)+'.png',new_img)
                    i += 1
                    flag.append("True")
                else:
                    new_img=image[y:y+h,x:x+w]
                    hh = int(h/2)
                    line1 = image[y:y+hh, x:x+w]
                    line1 = imutils.resize(line1, width=300)
                    line2 = image[y+hh:y+h, x:x+w]
                    cv2.imwrite('./images/temp_license_plate/'+str(i)+'.png',new_img)
                    cv2.imwrite('./images/temp_license_plate/'+str(i)+'.1.png',line1)
                    cv2.imwrite('./images/temp_license_plate/'+str(i)+'.2.png',line2)
                    i += 1
                    flag.append("False")
        for j in range(0, i, 1):
            if(flag[j] == True):
                Cropped_loc = "./images/temp_license_plate/" + str(j) + ".png"
                plate = pytesseract.image_to_string(Cropped_loc, lang='eng', config ='--oem 3 --psm 7 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789')
                if(5 <= len(plate) <=9):
                    return plate
            else:
                Cropped_loc = "./images/temp_license_plate/" + str(j) + ".png"
                line1 = "./images/temp_license_plate/" + str(j) + ".1.png"
                line2 = "./images/temp_license_plate/" + str(j) + ".2.png"
                plate1 = pytesseract.image_to_string(line1, lang='eng', config ='--oem 3 --psm 7 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789')
                plate2 = pytesseract.image_to_string(line2, lang='eng', config ='--oem 3 --psm 7 -c tessedit_char_whitelist=0123456789')
                plate = plate1+plate2
                plate = plate.replace('\n', "")
                if(5 <= len(plate) <=9):
                    return plate